In [1]:
import motile
import networkx as nx
from LineageTree import lineageTree
import utils
import os
path = r'/localscratch/DL4MIA_2024/project/data/tracks'
fname = r'GN692_125kPa002_20231017-142851.xml'
image_path = r'/group/dl4miacourse/TRNs_calcium/...'

file = os.path.join(path, fname)
lt = lineageTree(file, file_type='trackmate')

nx_graph = utils.to_motile(lt, crop=100, max_dist=20)

graph = motile.TrackGraph(nx_graph)

Extracting candidate edges
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


100%|██████████| 99/99 [00:00<00:00, 33242.82it/s]


In [4]:
import motile

# create a motile solver
solver = motile.Solver(graph)

In [5]:
from motile.costs import EdgeDistance

solver.add_costs(
    EdgeDistance(
        weight=1,
        position_attribute="pos",
        constant=-20))


In [8]:
from motile.constraints import MaxParents, MaxChildren

solver.add_constraints(MaxParents(1))
solver.add_constraints(MaxChildren(1))

In [11]:
from motile.costs import Appear

solver.add_costs(Appear(constant=1.0))

In [12]:
solution = solver.solve()

Could not create Gurobi backend: Gurobi error in ilpy/impl/solvers/GurobiBackend.cpp:22: PIP license can only be used from gurobipy interface


TODO:
add results to napari tracks layer and/or points layer and start viewer
Add raw data to image layer